### Chapter10.스파크 SQL

#### 10.1 SQL이란
- 구조적 질의 언어
- 스파크는 ANSISQL:2003의 일부 구현

#### 10.2 빅데이터와 SQL: 아파치 하이브

#### 10.3 빅데이터와 SQL: 스파크 SQL
- 스파크 2.0 버전 -> ANSI-SQL과 HiveQL을 모두 지원하는 자체 개발된 SQL 파서 포함
- 스파크 SQL은 DataFrame과의 호환성 좋음
    - SQL로 데이터 죄회, DataFrame으로 변환 후 스파크의 MLlib 활용 머신러닝 알고리즘 수행 결과를 다른 데이터 소스에 저장하는 등의 과정을 가능하게 함.
- OLAP 데이터베이스로 동작

#### 10.3.1 스파크와 하이브의 관계
- 하이브 메타스토어를 사용

#### 10.4 스파크 SQL 쿼리 실행 방법
#### 10.4.1 스파크 SQL CLI
- 스파크 SQL CLI는 쓰리프트 JDBC 서버와 통신 불가

#### 10.4.2 스파크의 프로그래밍 SQL 인터페이스
- SparkSession 객체의 sql 메서드 사용
- 처리된 결과는 DataFrame을 반환

In [33]:
import pyspark
# from pyspark.sql import SparkSession
# spark = SparkSession \
#     .builder \
#     .appName("SparkByExamples.com") \
#     .enableHiveSupport() \
#     .getOrCreate()

In [4]:
spark.sql("SELECT 1 + 1").show()

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+



In [12]:
spark.read.json("../../assets/exercises/week10/2015-summary.json")\
    .createOrReplaceTempView("some_sql_view")

In [13]:
spark.sql("""
    SELECT DEST_COUNTRY_NAME, sum(count)
    FROM some_sql_view GROUP BY DEST_COUNTRY_NAME
""")\
    .where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10")\
    .count()

12

### 10.4.3 스파크 SQL 쓰리프트 JDBC/ODBC 서버
- 자바 데이터베이스 연결 인터페이스를 제공
- 사용자나 원격 프로그램은 스파크 SQL을 실행하기 위해 이 인터페이스로 스파크 드라이버에 접속 가능
- 태블로 같은 비즈니스 인텔리전스 스포트웨어 이용하여 스파크에 접속하는 혀애가 대표적인 활용 사례

### 10.5 카탈로그
- 스파크SQL에서 가장 높은추상화 단계 카탈로그(catalog)
- 메타데이터,데이터베이스,테이블,함수,뷰에 대한 정보를 추상화
- 테이블,데이터베이스 그리고 함수를 조회

### 10.6 테이블
- 스파크에서 테이블을 생성하면 default 데이터베이스에 등록
- 스파크 2.x 버전에서 테이블은 항상 데이터를 가지고 있다. -> 임시 테이블의 개념이 없으며 데이터를 가지지않은 뷰만 존재

### 10.6.1 스파크 관리형 테이블
- 관리형 테이블 / 외부 테이블
- DataFrame의 saveAsTable 메서드
    - 스파크가 관련된 모든 정보를 추적할 수 있는 관리형 테이블을 만들 수 있음
    - 테이블을 읽고 데이터를 스파크 포맷으로 변환한 후 새로운 경로에 저장
    - spark.sql.warehouse.dir 속성에 디렉토리 경로를 지정 가능
    
### 10.6.2 테이블 생성하기
- SQL에서 전체 데이터소스 API를 재사용할 수 있는 기능 지원
- 스파크는 실행 즉시 테이블을 생성
- USING / STORED AS
    - 포맷 지정 없을 경우 스파크는 기본적으로 하이브 SerDe 설정 사용(스파크의 자체 직렬화보다 훨씬 느림)
    - 하이브 사용자는 STORED AS 구문으로 하이브 테이블 생성 가능

In [15]:
spark.sql("""CREATE TABLE flights_v2 (
    DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
    USING JSON OPTIONS (path '../../assets/exercises/week10/2015-summary.json')""")

23/04/16 03:42:42 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider JSON. Persisting data source table `default`.`flights_v2` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [18]:
# spark.sql("SELECT * FROM flights_v2").show(5)

### 10.6.3 외부 테이블 생성하기
- 외부 테이블 생성: CREATE EXTERNAL TABLE 구문

In [20]:
spark.sql("""CREATE EXTERNAL TABLE hive_flights(
    DEST_CouNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '../../assets/exercises/week10/flight-data-hive/'""")

DataFrame[]

In [22]:
spark.sql("SELECT * FROM hive_flights").show(5)

+-----------------+-------------------+-----+
|DEST_CouNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
+-----------------+-------------------+-----+



### 10.6.4 테이블에 데이터 삽입하기
- 표준 SQL 문법을 따른다
- 특정 파티션에만 저장하고 싶은 경우 파티션 명세를 추가 가능

### 10.6.5 테이블 메타데이터 확인하기
- DESCRIBE 구문 사용
- 테이블의 메타데이터 정보를 반환

### 10.6.6 테이블 메타데이터 갱신하기
- REFRESH TABLE: 테이블과 관련된 모든 캐싱된 항목(기본적으로 파일)을 갱신
- REPAIR TABLE: 카타로그에서 관리하는 테이블의 파티션 정보를 갱신
    - 새로운 파티션 정보를 수집하는 데 초점
    
### 10.6.7 테이블 제거하기
- delete 불가 / drop만 가능

`DROP TABLE [IF EXISTS]`

- 외부 테이블 제거: 데이터는 삭제되지 않지만, 더는 외부 테이블명을 이용해 데이터 조회 불가

### 10.6.8 테이블 캐싱하기
- DataFrameㅇ과 같이 테이블을 캐시하거나 캐시에서 삭제 가능

`CACHE TABLE [tablename]`

In [24]:
spark.sql("DESCRIBE TABLE flights")

23/04/16 03:49:10 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[col_name: string, data_type: string, comment: string]

### 10.7 뷰
- 단순 쿼리 실행 계획
- 쿼리 로직을 체계화 및 재사용 편리하게 가능
- 데이터베이스에 설정하는 전역 뷰나 세션별 뷰가 될 수 있음

### 10.7.1 뷰 생성하기
- 신규 경로에 모든 데이터를 다시 저장하는 대신 단순하게 쿼리 시점에 데이터소스에 트랜스포메이션을 수행
    - 대규모 GROUP BY, ROLLUP 같은 트랜스포메이션
- 임시 뷰 생성 가능

### 10.7.2 뷰 제거하기
`DROP VIEW IF EXISTS just_usa_view;`

In [44]:
flights = spark.read.format("json").load("../../assets/exercises/week10/2015-summary.json")

In [45]:
flights.where("dest_country_name = 'United States'").createOrReplaceTempView("just_usa_df")
just_usa_df.selectExpr("*").explain

<bound method DataFrame.explain of DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]>

In [39]:
spark.sql("EXPLAIN SELECT * FROM just_usa_view").show()

+--------------------+
|                plan|
+--------------------+
|== Physical Plan ...|
+--------------------+



In [51]:
just_usa_df.explain(mode='formatted')

== Physical Plan ==
* Filter (2)
+- Scan json  (1)


(1) Scan json 
Output [3]: [DEST_COUNTRY_NAME#199, ORIGIN_COUNTRY_NAME#200, count#201L]
Batched: false
Location: InMemoryFileIndex [file:/Users/hjb/Developer/spark-the-definitive-guide-study/assets/exercises/week10/2015-summary.json]
PushedFilters: [IsNotNull(DEST_COUNTRY_NAME), EqualTo(DEST_COUNTRY_NAME,United States)]
ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>

(2) Filter [codegen id : 1]
Input [3]: [DEST_COUNTRY_NAME#199, ORIGIN_COUNTRY_NAME#200, count#201L]
Condition : (isnotnull(dest_country_name#199) AND (dest_country_name#199 = United States))




In [52]:
type(just_usa_df)

pyspark.sql.dataframe.DataFrame

In [47]:
flights.where("DEST_COUNTRY_NAME = 'United States'").explain(mode='formatted')

== Physical Plan ==
* Filter (2)
+- Scan json  (1)


(1) Scan json 
Output [3]: [DEST_COUNTRY_NAME#247, ORIGIN_COUNTRY_NAME#248, count#249L]
Batched: false
Location: InMemoryFileIndex [file:/Users/hjb/Developer/spark-the-definitive-guide-study/assets/exercises/week10/2015-summary.json]
PushedFilters: [IsNotNull(DEST_COUNTRY_NAME), EqualTo(DEST_COUNTRY_NAME,United States)]
ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>

(2) Filter [codegen id : 1]
Input [3]: [DEST_COUNTRY_NAME#247, ORIGIN_COUNTRY_NAME#248, count#249L]
Condition : (isnotnull(DEST_COUNTRY_NAME#247) AND (DEST_COUNTRY_NAME#247 = United States))




### 10.8 데이터베이스
- 스파크에서 실행하는 모든 SQL 명령문은 사용 중인 데이터베이스 범위에서 실행
    - 동료와 동일한 컨텍스트나 세션을 공유하는 경우 혼란을 야기할 수 있으므로 사용할 데이터베이스를 반드시 설정해야 함
- 전체 데이터 베이스 목록 확인

`SHOW DATABASE`

### 10.8.1 데이터 베이스 생성하기
`CREATE DATABASE some_db`

### 10.8.2 데이터 베이스 설정하기
`USE some_db`

- 현재 어떤 데이터베이스 사용중인지 확인
`SELECT current_database()`

### 10.8.3 데이터 베이스 제거하기
`DROP TABLE IF EXISTS some_db`

### 10.9 select 구문
### 10.9.1 case when then 구문

### 10.10 고급 주제
### 10.10.1 복합 데이터 타입
- 스파크 SQL에는 구조체, 리스트, 맵 세 가지 핵심 복합 데이터 타입이 존재
- 구조체
    - 스파크에서 중첩 데이터를 생성하거나 쿼리하는 방법을 제공
    - 여러 컬럼이나 표현식을 괄호로 묶어서 생성 가능
    - 개별 컬럼 조회 가능: 점(dot) 사용
- 리스트
    - collect_list 함수 / collect_set 함수 사용 -> 집계 함수
    - 위 두 함수는 집계 연산 시에만 사용 가능
### 10.10.2 함수
- 스파크 SQL이 제공하는 전체 함수 목록을 확인하려면 SHOW FUNCTIONS 구문 사용

### 10.10.3 서브쿼리
- 쿼리 안에 쿼리를 지정 가능
- 상호연관 서브쿼리: 서브쿼리의 정보를 보완하기 위해 쿼리의 외부 범위에 있는 일부 정보 사용 가능
- 비상호연관 서브쿼리: 외부 범위에 있는 정보를 사용하지 않음.
- 조건절 서브쿼리: 값에 따라 필터링 가능

In [60]:
flights = spark.read.format("json").load("../../assets/exercises/week10/2015-summary.json").createOrReplaceTempView("flights")

In [62]:
spark.sql("SELECT DEST_COUNTRY_NAME FROM flights GROUP BY DEST_COUNTRY_NAME ORDER BY sum(COUNT) DESC LIMIT 5").show()

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|           Canada|
|           Mexico|
|   United Kingdom|
|            Japan|
+-----------------+



In [65]:
spark.sql("""SELECT * FROM flights 
    WHERE ORIGIN_COUNTRY_NAME IN (
        SELECT DEST_COUNTRY_NAME FROM flights GROUP BY DEST_COUNTRY_NAME ORDER BY sum(COUNT) DESC)
    LIMIT 5""").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [73]:
spark.sql("""SELECT * FROM flights f1
    WHERE EXISTS (
        SELECT 1 FROM flights f2
        WHERE f1.DEST_COUNTRY_NAME = f2.ORIGIN_COUNTRY_NAME)
    AND EXISTS (
        SELECT 1 FROM flights f2
        WHERE f2.DEST_COUNTRY_NAME = f1.ORIGIN_COUNTRY_NAME)
    """).show()
# EXISTS 키워드는 서브쿼리에 값이 존재하면 true를 반환

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|             Romania|   15|
|       United States|             Croatia|    1|
|       United States|             Ireland|  344|
|               Egypt|       United States|   15|
|       United States|               India|   62|
|       United States|           Singapore|    1|
|       United States|             Grenada|   62|
|          Costa Rica|       United States|  588|
|             Senegal|       United States|   40|
|       United States|        Sint Maarten|  325|
|       United States|    Marshall Islands|   39|
|              Guyana|       United States|   64|
|               Malta|       United States|    1|
|            Anguilla|       United States|   41|
|             Bolivia|       United States|   30|
|       United States|            Paraguay|    6|
|Turks and Caicos ...|       United States|  230|


### 10.11 다양한 기능
### 10.11.1 설정
<img src="../../assets/presentations/week10/chapter10_1.jpeg" width="1000px" height="250px" title="catalyst_optimizer"/>
<img src="../../assets/presentations/week10/chapter10_2.jpeg" width="1000px" height="250px" title="catalyst_optimizer"/>


### 10.11.2 SQL에서 설정값 지정하기
- 스파크 SQL과 관련된 설정만 가능
- 셔플 파티션 수 지정 `SET spark.sql.shuffle.partitions=20`